In [2]:
from pathlib import Path
import os
import librosa
import numpy as np
import soundfile as sf
import noisereduce as nr
import scipy.signal as signal

In [3]:
def denoise_spectral_gate(
    waveform,
    sample_rate,
    stationary = True,
    noise_duration = 0.5,
    n_fft = 4096,
    hop_length = 512,
    time_mask_smooth_ms= 80,
    freq_mask_smooth_hz = 400,
    prop_decrease = 0.8,
):
    """Spectral gating noise reduction"""
    noise_samples = int(noise_duration * sample_rate)
    y_noise = waveform[:noise_samples]
    return nr.reduce_noise(
        waveform,
        sample_rate,
        stationary=stationary,
        y_noise=y_noise,
        n_fft=n_fft,
        hop_length=hop_length,
        time_mask_smooth_ms=time_mask_smooth_ms,
        freq_mask_smooth_hz=freq_mask_smooth_hz,
        prop_decrease=prop_decrease,
    )

def normalize_and_center(audio) -> np.ndarray:
    """Peak normalization + zero-mean"""
    max_amp = np.max(np.abs(audio))
    normalized = audio / max_amp if max_amp > 0 else audio
    return normalized - normalized.mean()

def apply_pre_emphasis(audio, alpha= 0.97):
    """Pre-emphasis filter (Notebook 3)."""
    pre_emphasis = np.array([1.0, -alpha])
    return signal.lfilter(pre_emphasis, 1, audio)

In [6]:
INPUT_ROOT = Path(r"X:\数据集\DeepShip\data_preprocessing\data_new")
OUTPUT_ROOT = Path(r"X:\数据集\DeepShip\data_preprocessing\data_new_preprocessed")
CATEGORIES = ["Cargo", "Passengership", "Tanker", "Tug"]

for category in CATEGORIES:
    in_dir = INPUT_ROOT / category
    out_dir = OUTPUT_ROOT / category
    out_dir.mkdir(parents=True, exist_ok=True)

    for wav_path in sorted(in_dir.glob("*.wav")):
        out_path = out_dir / wav_path.name

        if out_path.exists():
            print(f"{category}: {wav_path.name} already done, skip.")
            continue

        waveform, sr = librosa.load(wav_path, sr=None)

        denoised = denoise_spectral_gate(waveform, sr)
        normalized = normalize_and_center(denoised)
        emphasized = apply_pre_emphasis(normalized)

        sf.write(out_dir / wav_path.name, emphasized, sr)
        print(f"{category}: {wav_path.name} processed.")

Cargo: 0_1.wav already done, skip.
Cargo: 0_10.wav already done, skip.
Cargo: 0_100.wav already done, skip.
Cargo: 0_101.wav already done, skip.
Cargo: 0_102.wav already done, skip.
Cargo: 0_103.wav already done, skip.
Cargo: 0_104.wav already done, skip.
Cargo: 0_105.wav already done, skip.
Cargo: 0_106.wav already done, skip.
Cargo: 0_107.wav already done, skip.
Cargo: 0_108.wav already done, skip.
Cargo: 0_109.wav already done, skip.
Cargo: 0_11.wav already done, skip.
Cargo: 0_110.wav already done, skip.
Cargo: 0_111.wav already done, skip.
Cargo: 0_12.wav already done, skip.
Cargo: 0_13.wav already done, skip.
Cargo: 0_14.wav already done, skip.
Cargo: 0_15.wav already done, skip.
Cargo: 0_16.wav already done, skip.
Cargo: 0_17.wav already done, skip.
Cargo: 0_18.wav already done, skip.
Cargo: 0_19.wav already done, skip.
Cargo: 0_2.wav already done, skip.
Cargo: 0_20.wav already done, skip.
Cargo: 0_22.wav already done, skip.
Cargo: 0_24.wav already done, skip.
Cargo: 0_25.wav al